In [1]:
import os
import pandas as pd
import polars as pl
pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

%load_ext autoreload
%autoreload 2

os.getcwd()

'/home/justinxu/esgpt/ESGPTTaskQuerying/to_organize'

### In-Hospital Mortality

Query

In [2]:
from esgpt_task_querying import main

/home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
config_path = 'test_configs/abnormal_lab.yaml'

data_path = '../MIMIC_ESD_new_schema_08-31-23-1'

In [8]:
main.query_task(config_path, data_path)

Updating config.save_dir from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_new_schema_08-31-23-1 to ../MIMIC_ESD_new_schema_08-31-23-1
Loading events from ../MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet...


Loading dynamic_measurements from ../MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet...
Loading config...

Generating predicate columns...

Added predicate column is_admission.
Added predicate column is_discharge.
Added predicate column is_death.
Added predicate column is_lab.
Added predicate column is_abnormal_lab_num.
Added predicate column is_heart_rate.
Added predicate column is_discharge_or_death.
Added predicate column is_heart_rate_and_abnormal_lab_num.

Building tree...
trigger
┗━━ gap
    ┗━━ target


Querying...
Querying subtree rooted at gap...


ComputeError: input data is not sorted

Validation